### 컴퓨터 비전 : 객체 탐지 개발 파이프라인

- 이 노트북은 **`객체 탐지` 개발 파이프라인 예제 실습**을 수행하는 노트북입니다.

##### YOLO를 활용한 객체 탐지 개발 파이프라인 예제
   1.  YOLO 학습용/검증용 데이터셋 전처리 (Cell 1개)
   2.  YOLO Fine-Tuning 및 저장 (Cell 1개)
   3.  Fine-tuning 한 모델의 성능 평가 (Cell 1개)

### 01. YOLO 학습용/검증용 데이터셋 전처리

ImageSets/Main의 trainval.txt와 test.txt 파일을 기반으로 학습용과 검증용 데이터를 분리합니다.

원본 이미지와 라벨 파일들을 YOLO 형식의 디렉토리 구조(images/train, labels/train, images/test, labels/test)로 재구성하여 이동시킵니다.

* 데이터셋 루트 경로 설정 및 원본 이미지(JPEGImages), 라벨(Annotations_txt) 폴더 지정
* 학습용과 검증용 타겟 디렉토리(images/train, images/test, labels/train, labels/test) 생성
* trainval.txt 파일을 읽어 해당 ID의 이미지와 라벨을 train 폴더로 이동
* test.txt 파일을 읽어 해당 ID의 이미지와 라벨을 test 폴더로 이동

In [2]:
# ============================================ 
# Cell 1: 데이터 전처리 실행 (선택사항)
# ============================================

# 주의: 이 셀은 kaggle Fashion Dataset이 존재하고 전처리를 진행하지 않은 경우에만 실행하세요.
# 이미 전처리 데이터로 dataset.zip의 압축을 풀어서 활용하셔도 됩니다.
# 이미 전처리된 데이터가 있다면 이 셀을 건너뛰고 Cell 2로 진행하세요.

#ImageSets/Main 내 trainval.txt와 text.txt 파일을 읽어 학습할 이미지/레이블 셋과 검증용 이미지/레이블 셋을 분리하여 파일 이동
#학습용 이미지 & 레이블 : images/train, labels/train
#검증용 이미지 & 레이블: images/test, images/test

import os
import shutil

# dataset root 경로 (작업 디렉토리에 맞게 수정)
ROOT        = "./dataset/fashion"
JPEG_DIR    = os.path.join(ROOT, "JPEGImages")      # 원본 이미지 폴더
LABELS_SRC  = os.path.join(ROOT, "Annotations_txt") # 원본 라벨 폴더
SETS_DIR    = os.path.join(ROOT, "ImageSets", "Main")
TRAIN_IDS   = os.path.join(SETS_DIR, "trainval.txt")
TEST_IDS    = os.path.join(SETS_DIR, "test.txt")

# 이동할 타겟 디렉토리
IMG_DST     = os.path.join(ROOT, "images")
LBL_DST     = os.path.join(ROOT, "labels")

# train/test 폴더 생성
for split in ("train", "test"):
    os.makedirs(os.path.join(IMG_DST, split), exist_ok=True)
    os.makedirs(os.path.join(LBL_DST, split), exist_ok=True)

def move_split(id_file, split):
    """
    id_file: trainval.txt 또는 test.txt 경로
    split:   "train" 또는 "test"
    """
    with open(id_file, "r", encoding="utf-8") as f:
        for line in f:
            idx = line.strip()
            if not idx:
                continue

            # 이미지 파일 이동
            src_img = os.path.join(JPEG_DIR, f"{idx}.jpg")
            dst_img = os.path.join(IMG_DST, split, f"{idx}.jpg")
            if os.path.exists(src_img):
                if not os.path.exists(dst_img):  # 목적지에 파일이 없을 때만 이동
                    shutil.move(src_img, dst_img)
                else:   # Source 파일과 Dest 파일이 모두 존재하면 Skip
                    print(f"Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) {idx}.jpg")
            else:
                if not os.path.exists(dst_img):  # 목적지에도 없다면 경고
                    print(f"Warning: 이미지 없음 {src_img}")
                else:   # Dest 파일이 모두 존재하면 Skip
                    print(f"Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) {idx}.jpg")

            # 라벨 파일 이동
            src_lbl = os.path.join(LABELS_SRC, f"{idx}.txt")
            dst_lbl = os.path.join(LBL_DST, split, f"{idx}.txt")
            if os.path.exists(src_lbl):
                if not os.path.exists(dst_lbl):  # 목적지에 파일이 없을 때만 이동
                    shutil.move(src_lbl, dst_lbl)
                else:   # Source 파일과 Dest 파일이 모두 존재하면 Skip
                    print(f"Skip: 이미 파일이 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) {idx}.txt")
            else:
                if not os.path.exists(dst_lbl):  # 목적지에도 없다면 경고
                    print(f"Warning: 라벨 없음 {src_lbl}")
                else:   # Dest 파일이 모두 존재하면 Skip
                    print(f"Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) {idx}.txt")

# 1) 학습용(trainval.txt) → images/train, labels/train
move_split(TRAIN_IDS, "train")
print("train split 완료")

# 2) 검증용(test.txt) → images/test, labels/test
move_split(TEST_IDS, "test")
print("test split 완료")

Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 168468.jpg
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 168468.txt
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 20751.jpg
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 20751.txt
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 184291.jpg
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 184291.txt
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 63239.jpg
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 63239.txt
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 110606.jpg
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 110606.txt
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 31187.jpg
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 31187.txt
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 103194.jpg
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 103194.txt
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터셋입니다.) 137927.jpg
Skip: 이미 이동됨 (datasets.zip 파일은 이미 전처리가 수행된 데이터

### 02. YOLO Fine-Tuning 및 저장

사전학습된 YOLOv8 nano 모델(`yolov8n.pt`)을 커스텀 패션 데이터셋으로 Fine-Tuning합니다.

GPU/CPU 환경을 자동 감지하고 30 에포크 동안 학습을 진행하여 최적화된 가중치를 생성합니다.

* PyTorch와 Ultralytics YOLO 라이브러리 임포트 및 CUDA GPU 사용 가능 여부에 따른 디바이스 설정
* COCO 데이터셋으로 사전학습된 YOLOv8 nano 모델 로드
* 커스텀 패션 데이터셋(fashion.yaml)으로 10 에포크, 배치 크기 8, 이미지 크기 640×640 설정하여 학습 실행
* 학습 완료 후 실험 결과 디렉토리와 최적 가중치(best.pt) 파일 경로 출력 및 확인

In [3]:
# ============================================ 
# Cell 2: YOLO Fine-Tuning 및 저장
# ============================================

import torch                   # PyTorch: 딥러닝 연산 및 GPU 사용을 위한 핵심 라이브러리
from ultralytics import YOLO   # Ultralytics YOLOv8 API: 객체 탐지 모델을 간편하게 사용
import os                      # 운영체제(OS) 경로 조작을 위한 표준 모듈
from pathlib import Path

# ─────────────────────────────────────────────────────────────────────────────
# 1) 디바이스 설정
#    torch.cuda.is_available()를 사용해 현재 환경에 CUDA GPU가 있는지 판단합니다.
#    GPU가 있으면 'cuda:0' (첫 번째 GPU), 없으면 'cpu'를 사용하도록 합니다.
#    이렇게 하면 같은 코드로 GPU/CPU 양쪽 환경 모두에서 동작합니다.
# ─────────────────────────────────────────────────────────────────────────────
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")  # 실제로 선택된 디바이스 정보를 출력해 확인

# ─────────────────────────────────────────────────────────────────────────────
# 2) 사전학습된 YOLOv8n 모델 로드
#    "yolov8n.pt"는 Ultralytics에서 제공하는 YOLOv8 Nano 모델의 가중치 파일로,
#    COCO 데이터셋(80개 클래스)으로 미리 학습되어 있습니다.
#    이 가중치를 불러오면 객체 탐지의 기본 특성(엣지, 모양 등)을 이미 학습한 상태로 시작할 수 있습니다.
# ─────────────────────────────────────────────────────────────────────────────
model = YOLO("yolov8n.pt")

# ─────────────────────────────────────────────────────────────────────────────
# 3) 모델 학습(Fine-Tuning) 실행
#    model.train() 호출 시 주요 인자:
#      - data  : "dataset/fashion.yaml"
#                  학습/검증 이미지 경로(path, train, val)와 클래스 목록(names)이 정의된 YAML 파일
#      - epochs: 1
#                  테스트를 위해 전체 데이터셋을 1번 반복 학습합니다. 데이터 양이나 자원에 맞춰 조정 가능합니다.
#      - batch : 8
#                  한 번에 GPU 메모리에 올려 처리할 이미지 수입니다. 메모리 용량에 맞춰 변경하세요.
#      - imgsz : 640
#                  입력 이미지 크기를 640×640 픽셀로 맞춥니다. (letterbox 방식으로 패딩하여 종횡비 유지)
#                  (height, width) 튜플로 다른 비율도 지정 가능합니다.
#      - device: 위에서 결정한 'cuda:0' 또는 'cpu'
#
#    이 호출 하나로 학습(train)과 검증(val)이 자동으로 수행되며,
#    매 에포크(epoch)마다 손실, mAP, Precision, Recall 등의 지표가 콘솔에 출력됩니다.
# ─────────────────────────────────────────────────────────────────────────────
results = model.train(
    data="dataset/fashion.yaml",
    epochs=1,
    batch=8,
    imgsz=640,
    device=device
)

# ─────────────────────────────────────────────────────────────────────────────
# 4) 학습 완료 후 최적 가중치(best weights) 경로 확인
#    results.save_dir 속성에는 이번 실험의 결과가 저장된 디렉토리(Path 객체)가 들어 있습니다.
#    일반적으로 runs/detect/trainX/ 형태로 생성되며, 그 하위 weights/best.pt에 최적 가중치가 저장됩니다.
# ─────────────────────────────────────────────────────────────────────────────
runs_dir = Path("runs/detect")
run_dir = sorted(runs_dir.glob("train*"), key=os.path.getmtime)[-1]

print("Run directory:", run_dir)  # 실험 결과 폴더 위치 확인

best_weights = os.path.join(run_dir, "weights", "best.pt")
print("Training complete. Best weights saved to:", best_weights)


Using device: cuda:0
Ultralytics 8.3.155  Python-3.10.18 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6141MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/fashion.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, p

100%|██████████| 755k/755k [00:00<00:00, 4.71MB/s]

Overriding model.yaml nc=80 with nc=10



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

100%|██████████| 5.35M/5.35M [00:01<00:00, 4.99MB/s]


...

Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1       1.1G      1.384      2.656      1.402         10        640: 100%|██████████| 269/269 [00:30<00:00,  8.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:04<00:00,  7.67it/s]


                   all        537       2035      0.556      0.614       0.59      0.364

1 epochs completed in 0.010 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.155  Python-3.10.18 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6141MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:03<00:00,  9.22it/s]


                   all        537       2035      0.555      0.614      0.591      0.364
              sunglass         82         82      0.194      0.329      0.157     0.0418
                   hat         77         77       0.53      0.143       0.36      0.149
                jacket        179        181      0.771      0.713      0.793      0.544
                 shirt        361        366      0.668      0.609      0.668      0.435
                 pants        114        114      0.737      0.868      0.895      0.646
                shorts        107        107      0.454      0.822      0.673      0.436
                 skirt        186        186      0.492      0.785      0.533      0.382
                 dress        128        128      0.523      0.351      0.456      0.326
                   bag        274        274      0.505      0.726       0.62      0.298
                  shoe        520        520      0.674      0.798       0.75      0.382
Speed: 0.2ms preproce

### 03. Fine-tuning 한 모델의 성능 평가

학습이 완료된 모델(`best.pt`)을 로드하여 검증 데이터셋으로 성능을 평가합니다.

mAP, Precision, Recall 등의 주요 지표를 계산하고 출력하여 모델의 객체 탐지 성능을 정량적으로 분석합니다.

* PyTorch와 Ultralytics YOLO 라이브러리 임포트 및 CUDA GPU 사용 가능 여부에 따른 디바이스 설정
* 학습된 최적 가중치(best.pt) 파일을 경로 지정하여 YOLOv8 모델로 로드
* 패션 데이터셋으로 검증 수행하여 DetMetrics 객체에서 성능 지표 추출
* AP50, mAP@0.50-0.95, Precision, Recall 값을 계산하고 포맷팅하여 콘솔 출력

In [4]:
# ============================================ 
# Cell 3: Fine-tuning 한 모델의 성능 평가
# ============================================

import torch                   # PyTorch: 모델 학습·추론 및 GPU 사용을 위한 핵심 라이브러리
from ultralytics import YOLO   # Ultralytics YOLOv8 API: 객체 탐지 모델을 간편하게 불러오기
import os                      # 운영체제 경로 조작을 위한 표준 모듈

# ─────────────────────────────────────────────────────────────────────────────
# 1) 디바이스 설정
#    - torch.cuda.is_available(): CUDA 지원 GPU 사용 가능 여부 확인
#    - 'cuda:0': 첫 번째 GPU 디바이스 지정
#    - 'cpu'    : GPU가 없을 경우 CPU 사용
# ─────────────────────────────────────────────────────────────────────────────
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ─────────────────────────────────────────────────────────────────────────────
# 2) 학습된 모델(best.pt) 로드
#    - YOLO(best_weights) 호출 시 해당 가중치 파일로 모델 초기화
# ─────────────────────────────────────────────────────────────────────────────
model = YOLO(best_weights)
print(f"Loaded best weights from: {best_weights}")

# ─────────────────────────────────────────────────────────────────────────────
# 3) 검증 수행 (Validation)
#    - model.val() 호출로 YAML에 정의된 검증 데이터셋 평가 실행
#    - data: 'dataset/fashion.yaml' (경로·클래스 정보 포함)
#    - imgsz: 입력 이미지 크기(px), train과 동일하게 설정
#    - batch : 배치 크기, GPU 메모리에 맞춰 조절
#    - device: 위에서 설정한 디바이스
#    반환값 metrics는 DetMetrics 객체로 다양한 지표를 포함
# ─────────────────────────────────────────────────────────────────────────────
metrics = model.val(
    data="dataset/fashion.yaml",
    imgsz=640,
    batch=8,
    device=device
)

# ─────────────────────────────────────────────────────────────────────────────
# 4) 검증 지표 추출
#    DetMetrics 객체 내부 box 속성에는 Metric 인스턴스가 있고,
#    그 안에 AP, mAP, Precision, Recall 등이 저장돼 있습니다.
#
#    - all_ap      : 클래스별 AP(10개 IoU threshold)
#    - map         : IoU 0.50:0.95 평균 mAP (COCO mAP)
#    - p, r        : 클래스별 Precision, Recall 리스트
# ─────────────────────────────────────────────────────────────────────────────

# 4.1) AP50 (IoU=0.5) 값 추출 및 클래스 평균 계산
ap50_per_class = metrics.box.all_ap[:, 0]      # AP50 배열 (클래스 수,)
mAP50 = float(ap50_per_class.mean())           # 전체 클래스 평균 AP50

# 4.2) COCO mAP (IoU 0.50~0.95 평균 AP)
mAP50_95 = float(metrics.box.map)

# 4.3) Precision, Recall 클래스 평균 계산
precision = float(sum(metrics.box.p) / len(metrics.box.p))
recall    = float(sum(metrics.box.r) / len(metrics.box.r))

# ─────────────────────────────────────────────────────────────────────────────
# 5) 결과 출력
#    포맷팅하여 주요 지표를 콘솔에 출력
# ─────────────────────────────────────────────────────────────────────────────
print("Validation results:")
print(f"  mAP@0.5       : {mAP50:.4f}")
print(f"  mAP@0.50–0.95 : {mAP50_95:.4f}")
print(f"  Precision     : {precision:.4f}")
print(f"  Recall        : {recall:.4f}")


Using device: cuda:0
Loaded best weights from: runs\detect\train\weights\best.pt
Ultralytics 8.3.155  Python-3.10.18 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6141MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 301.792.7 MB/s, size: 34.8 KB)


val: Scanning C:\Users\SSAFY\Desktop\AI_특강\dataset\fashion\labels\test.cache... 537 images, 0 backgrounds, 0 corrupt: 100%|██████████| 537/537 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 68/68 [00:03<00:00, 18.59it/s]


                   all        537       2035      0.557      0.614       0.59      0.363
              sunglass         82         82      0.189      0.317      0.155     0.0416
                   hat         77         77      0.534      0.143      0.357      0.148
                jacket        179        181      0.782      0.713      0.793      0.544
                 shirt        361        366      0.671      0.607      0.667      0.435
                 pants        114        114       0.74      0.877      0.898      0.643
                shorts        107        107      0.457      0.826      0.674      0.436
                 skirt        186        186      0.493      0.785      0.533      0.383
                 dress        128        128      0.519      0.346      0.456      0.326
                   bag        274        274      0.506      0.726       0.62      0.297
                  shoe        520        520      0.675      0.798      0.749      0.382
Speed: 0.3ms preproce